## Copyright 2021 Antoine Simoulin.

<i>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

Icons made by <a href="https://www.flaticon.com/authors/freepik" title="Freepik">Freepik</a>, <a href="https://www.flaticon.com/authors/pixel-perfect" title="Pixel perfect">Pixel perfect</a>, <a href="https://www.flaticon.com/authors/becris" title="Becris">Becris</a>, <a href="https://www.flaticon.com/authors/smashicons" title="Smashicons">Smashicons</a>, <a href="https://www.flaticon.com/authors/srip" title="srip">srip</a>, <a href="https://www.flaticon.com/authors/adib-sulthon" title="Adib">Adib</a>, <a href="https://www.flaticon.com/authors/flat-icons" title="Flat Icons">Flat Icons</a> and <a href="https://www.flaticon.com/authors/dinosoftlabs" title="Pixel perfect">DinosoftLabs</a> from <a href="https://www.flaticon.com/" title="Flaticon"> www.flaticon.com</a></i>

# Exercice Models de langue

In [ ]:
%%capture

# Check environment
if 'google.colab' in str(get_ipython()):
  IN_COLAB = True
else:
  IN_COLAB = False

if IN_COLAB:
  # ⚠️ Execute only if running in Colab
  !pip install -q transformers==3.1.0
  !pip install -q tensorflow==2.0.0
  # then restart runtime environment

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

## Exercice 1: les modèles de langues de types N-gram

Dans ce premier exercice, nous allons implémenter un modèle de langue de type N-gram pour construire un **système d'auto-complétion**.

Comme nous l'avons vu en cours, les modèles de langues n-grams cherchent à estimer la probabilité conditionnelle d'un mot $t$ dans la phrase étant donné les $n$ mots précédents $w_{t-1}, w_{t-2} \cdots w_{t-n}$ : 

$$ P(w_t | w_{t-1}\dots w_{t-n}) \tag{1}$$

On estime cette probabilité avec $\hat{P}$ en comptant les occurrences des sequences de mots dans les données d'entrainement :

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

Avec $C(\cdots)$ le nombre d'occurrences d'une séquence de mots donnée. En pratique, le dénominateur peut être nul. On va ajouter un paramètre de smoothing. On ajoute une constate $k$ au numérateur et $k \times |V|$ au dénominateur avec $|V|$ la taille du vocabulaire. On a donc :

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$

Si on a un n-grams qui n'apparait pas, l'équation (3) devient donc $\frac{1}{|V|}$.

In [ ]:
sentences = [['je', 'suis', 'en', 'vacances'],
             ['je', 'vais', 'partir', 'à', 'la', 'réunion'],
             ['je', 'suis', 'en', 'réunion'],
             ['je', 'vais', 'partir', 'en', 'vacances']]

In [ ]:
unique_words = list(set(sentences[0] + sentences[1] + sentences[2] + sentences[3]))
unique_words

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Ecrire une fonction qui génère tous les n-grams d'une phrase avec n un paramètre de la fonction.</p>
</div>
<hr>

In [ ]:
# %load solutions/ngrams.py

def sentence_2_n_grams(sentences, n=3, start_token='<s>', end_token='</s>'):
    
    # TODO Complete function
    
    return n_grams_list

In [ ]:
unigram_counts = sentence_2_n_grams(sentences, 1)
print("Uni-gram:")
print(unigram_counts)

bigram_counts = sentence_2_n_grams(sentences, 2)
print("\nBi-gram:")
print(bigram_counts)

trigram_counts = sentence_2_n_grams(sentences, 3)
print("\nTri-gram:")
print(trigram_counts)

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Ecrire une fonction qui calcule la probabilité d'un mot en fonction des ngrams précédents.</p>
</div>
<hr>

In [ ]:
# %load solutions/estimate_proba.py
def estimate_probability(word, previous_n_gram,
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    
    # TODO, complete word probability

    probability = 0

    return probability

In [ ]:
word_1 = "je vais"
word_2 = "partir"
tmp_prob = estimate_probability(word_2, word_1, bigram_counts, trigram_counts, len(unique_words), k=1)

print("La probabilité du mot '{}' étant donné le précédent n-gram '{}' est : {:.3f}."
      .format(word_2, word_1, tmp_prob))

In [ ]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0,
                           start_token='<s>', end_token='</s>', unk_token='<unk>'):
    
    # On ajoute end_token et unk_token to the vocabulary
    # start_token ne peut pas apparaitre comme mot suivant donc pas besoin de l'ajouter
    vocabulary = vocabulary + [end_token, unk_token]
    vocabulary_size = len(vocabulary)
    
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability

    return probabilities

In [ ]:
next_word_proba = estimate_probabilities("je", unigram_counts, bigram_counts, unique_words, k=1)

for w, p in next_word_proba.items():
    print("La probabilité du mot '{}' étant donné le précédent n-gram '{}' est : {:.3f}."
          .format(w, 'je', p))

In [ ]:
estimate_probabilities("en", bigram_counts, trigram_counts, unique_words, k=1)

for w, p in next_word_proba.items():
    print("La probabilité du mot '{}' étant donné le précédent n-gram '{}' est : {:.3f}."
          .format(w, 'en', p))

In [ ]:
def make_count_matrix(n_plus1_gram_counts, vocabulary,
                      start_token='<s>', end_token='</s>', unk_token='<unk>'):
 
    vocabulary = vocabulary + [end_token, unk_token]
    vocabulary_size = len(vocabulary)
    
    # obtain unique n-grams
    n_grams = list(n_plus1_gram_counts.keys())
    
    row_index = {n_gram: i for i, n_gram in enumerate(n_grams)}
    col_index = {word: j for j, word in enumerate(vocabulary)}
    
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram
        word = n_plus1_gram.split()[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=[' '.join(ng.split()[0:-1]) for ng in n_grams], columns=vocabulary)
    return count_matrix

In [ ]:
sentences = [['je', 'suis', 'en', 'vacances'],
             ['je', 'vais', 'partir', 'à', 'la', 'réunion'],
             ['je', 'suis', 'en', 'réunion'],
             ['je', 'vais', 'partir', 'en', 'vacances']]

display(make_count_matrix(bigram_counts, unique_words))

In [ ]:
# Show trigram counts
display(make_count_matrix(trigram_counts, unique_words))

In [ ]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [ ]:
display(make_probability_matrix(bigram_counts, unique_words, k=1))

In [ ]:
display(make_probability_matrix(trigram_counts, unique_words, k=1))

In [ ]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    
    n = len(list(n_gram_counts.keys())[0].split()) 
    previous_n_gram = ' '.join(previous_tokens.split()[-n:])
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)

    suggestion = None
    max_prob = 0
    for word, prob in probabilities.items(): 
        if prob > max_prob: 
            suggestion = word            
            max_prob = prob    
    return suggestion, max_prob

In [ ]:
previous_tokens = "je vais"
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"Pour les tokens 'je vais',la suggestion est le mot '{tmp_suggest1[0]}' avec une probabilité de {tmp_suggest1[1]:.4f}.")

On peut calculer la perplexité pour évaluer le modèle. Cette dernière est donnée par :

$$ PP(W) =\sqrt[N]{ \prod_{t=n+1}^N \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4}$$

Avec $N$ la longueur de la phrase et $n$ la taille des n-grams (par exemple 2 dans le cas des bigrams). On cherche à minimiser la perplexité du modèle.

In [ ]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0,
                         start_token='<s>', end_token='</s>', unk_token='<unk>'):
    
    n = len(list(n_gram_counts.keys())[0].split()) 
    tokens = [start_token] + sentence + [end_token]
    N = len(tokens)
    
    product_pi = 1.0
  
    for t in range(n, N): 
        n_gram = tokens[t-n:t]
        word = tokens[t]
        
        probability = estimate_probability(word, ' '.join(n_gram), 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           len(unique_words), k=1)
        product_pi *= 1 / probability

    perplexity = product_pi**(1/float(N))
    
    return perplexity

In [ ]:
perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"La perplexité pour la première phrase du corpus est : {perplexity_train1:.4f}.")


perplexity_train1 = calculate_perplexity(['Tu' ,'pars', 'ou', 'en', 'vacances', '?'],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"La perplexité pour la phrase test est : {perplexity_train1:.4f}.")

## Exercice 2: les modèles de langues avec réseaux de neurones : GPT-2


J'ai entrainé un modèle GPT-2 <span class="badge badge-secondary">([Radford et al., 2019](#radford-2019))</span> sur 50M de phrases extraites du corpus OSCAR <span class="badge badge-secondary">([Suárez et al., 2019](#suarez-2019))</span>. Le modèle a ensuite été fine-tuné (on a continué l'entrainement) sur le Tome 2 de Harry Potter. Ce modèle est une architecture de réseaux de neurones assez connu pour les modèles de langues. Il permet de générer du texte de manière assez réaliste. On va utiliser la librairie `transformers` pour utiliser le modèle. **Vous devez récupérer les poids du modèles sur le moodle (fichier french-gpt2-hp)**.

In [ ]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("./french-gpt2-hp")
model = TFGPT2LMHeadModel.from_pretrained("./french-gpt2-hp", 
                                          pad_token_id=tokenizer.eos_token_id, from_pt=True)

In [ ]:
input_ids = tokenizer.encode(
    "Dans son mouvement, la queue du Basilic lui avait jeté le Choixpeau magique à la tête. ",
    return_tensors='tf')

In [ ]:
# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

In [ ]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

In [ ]:
input_ids = tokenizer.encode(
    "Assis un peu plus loin, Harry reconnut Gilderoy Lockhart, vêtu d'une robe de sorcier bleu-vert.",
    return_tensors='tf')

In [ ]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

## 📚 References

> <div id="radford-2019">Alec Radford, Jeffrey Wu, Rewon Child, David Luan and Dario Amodei. <a href=https://openai.com/blog/better-language-models/> Better Language Models and Their Implications.</a></div>

> <div id="suarez-2019">Suárez, Pedro Javier Ortiz, Benoît Sagot, and Laurent Romary. <a href=https://hal.inria.fr/hal-02148693> Asynchronous pipeline for processing huge corpora on medium to low resource infrastructures.</a> 7th Workshop on the Challenges in the Management of Large Corpora (CMLC-7). Leibniz-Institut für Deutsche Sprache, 2019.</div>